In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.tools import tool
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from IPython.display import JSON, display 

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0,
    max_output_tokens=2000,
)


E0000 00:00:1762680750.551017    9721 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
system_prompt = """
You are *Mathy*, a friendly AI math assistant 🤖.
- Use tools when necessary to solve problems.
- Explain reasoning briefly before giving the final answer.
- Respond clearly in Bahasa Indonesia.
"""

In [ ]:
@tool
def add_numbers(numbers: list[float]) -> float:
    """Menjumlahkan daftar angka."""
    return sum(numbers)

@tool
def subtract_numbers(a: float, b: float) -> float:
    """Mengurangkan angka kedua dari angka pertama."""
    return a - b

@tool
def multiply_numbers(numbers: list[float]) -> float:
    """Mengalikan semua angka dalam daftar."""
    result = 1
    for n in numbers:
        result *= n
    return result

@tool
def divide_numbers(a: float, b: float) -> float:
    """Membagi a dengan b. Tangani pembagian dengan nol."""
    if b == 0:
        return "Error: tidak bisa membagi dengan nol."
    return a / b


wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

@tool
def search_wikipedia(query: str) -> str:
    """Mencari informasi di Wikipedia berdasarkan query pengguna."""
    return wiki.run(query)


In [ ]:
print("Name: \n", add_numbers.name)
print("Description: \n", add_numbers.description) 
print("Args: \n", add_numbers.args) 

Name: 
 add_numbers
Description: 
 Menjumlahkan daftar angka.
Args: 
 {'numbers': {'items': {'type': 'number'}, 'title': 'Numbers', 'type': 'array'}}


In [12]:
tools=[add_numbers, subtract_numbers, multiply_numbers, divide_numbers, search_wikipedia]
       
for tool in tools:
    schema = {
   "name": tool.name,
   "description": tool.description,
   "parameters": tool.args_schema.schema() if tool.args_schema else {},
   "return": tool.return_type if hasattr(tool, "return_type") else None}
    display(JSON(schema))

/tmp/ipykernel_9721/2561180371.py:7: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  "parameters": tool.args_schema.schema() if tool.args_schema else {},


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

In [ ]:
agent = create_agent(
    model=llm,
    tools=[add_numbers, subtract_numbers, multiply_numbers, divide_numbers, search_wikipedia],
    system_prompt=system_prompt,
    debug=True 
    )

In [7]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Tambahkan 2, 3, dan 4, lalu kalikan 0"}
    ]
})

print("=== FULL RAW RESULT ===")
print(result)
print("\n=== FINAL ANSWER ===")
print(result["messages"][-1].content)

[values] {'messages': [HumanMessage(content='Tambahkan 2, 3, dan 4, lalu kalikan 0', additional_kwargs={}, response_metadata={}, id='1b160409-6b53-4bce-9d0f-6656228c4d44')]}
[updates] {'model': {'messages': [AIMessage(content='Tentu, saya akan bantu menghitungnya.\n\nPertama, saya akan menjumlahkan 2, 3, dan 4.', additional_kwargs={'function_call': {'name': 'add_numbers', 'arguments': '{"numbers": [2.0, 3.0, 4.0]}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--3dc51dd7-0e19-4cad-9c09-fbc161f04779-0', tool_calls=[{'name': 'add_numbers', 'args': {'numbers': [2.0, 3.0, 4.0]}, 'id': '41462483-a797-4b35-80d2-de7a0e3b6314', 'type': 'tool_call'}], usage_metadata={'input_tokens': 296, 'output_tokens': 51, 'total_tokens': 347, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanM

In [37]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Siapa Prabowo Subianto?"}
    ]
})

print("=== FULL RAW RESULT ===")
print(result)
print("\n=== FINAL ANSWER ===")
print(result["messages"][-1].content)

[values] {'messages': [HumanMessage(content='Siapa Prabowo Subianto?', additional_kwargs={}, response_metadata={}, id='e9972081-16aa-48fd-a7ce-4bfaf8a6b139')]}
[updates] {'model': {'messages': [AIMessage(content='Tentu, saya akan mencari informasi tentang Prabowo Subianto di Wikipedia.', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query": "Prabowo Subianto"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--22e88fab-00f9-4296-b7db-e771067d66ce-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Prabowo Subianto'}, 'id': 'a9025c6e-319b-4627-a449-40485fc36f6c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 287, 'output_tokens': 37, 'total_tokens': 324, 'input_token_details': {'cache_read': 0}})]}}
[values] {'messages': [HumanMessage(content